# AutoGen Základní Ukázka

V tomto příkladu kódu použijete [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework k vytvoření základního agenta.

Cílem této ukázky je ukázat vám kroky, které později použijeme v dalších příkladech kódu při implementaci různých agentních vzorů.


## Importujte potřebné balíčky Pythonu


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Vytvoření klienta

V tomto příkladu použijeme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pro přístup k LLM.

`model` je definován jako `gpt-4o-mini`. Zkuste změnit model na jiný dostupný v marketplace GitHub Models a podívejte se na různé výsledky.

Jako rychlý test spustíme jednoduchý dotaz - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definování agenta

Nyní, když jsme nastavili `client` a ověřili, že funguje, vytvoříme `AssistantAgent`. Každému agentovi lze přiřadit:  
**name** - Krátké jméno, které bude užitečné při odkazování v procesech s více agenty.  
**model_client** - Klient, kterého jste vytvořili v předchozím kroku.  
**tools** - Dostupné nástroje, které agent může použít k dokončení úkolu.  
**system_message** - Metaprompt, který definuje úkol, chování a tón LLM.  

Můžete změnit systémovou zprávu a zjistit, jak LLM reaguje. `tools` probereme v lekci č. 4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Spuštění agenta

Níže uvedená funkce spustí agenta. Používáme metodu `on_message` k aktualizaci stavu agenta novou zprávou.

V tomto případě aktualizujeme stav novou zprávou od uživatele, která zní: `"Naplánuj mi skvělou slunečnou dovolenou"`.

Můžete změnit obsah zprávy a zjistit, jak LLM reaguje odlišně.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Upozornění**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o co největší přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za závazný zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
